In [ ]:
# beautiful soup는 webscrapping을 하는 tool 
from bs4 import BeautifulSoup

html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

soup = BeautifulSoup(html_doc, 'html.parser')
print(soup.prettify())


In [ ]:
print(soup.title)

In [ ]:
print(soup.p)

In [ ]:
print(soup.find('p','title'))

In [ ]:
# response는 200이 나오면 정상 
import requests

base_url = 'https://bjpublic.tistory.com/category/%EC%A0%84%EC%B2%B4%20%EC%B6%9C%EA%B0%84%20%EB%8F%84%EC%84%9C?page=1'
response = requests.get(base_url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    # html = response.text

In [ ]:
soup.find('div')

In [ ]:
import asyncio
import aiohttp
import nest_asyncio
nest_asyncio.apply()
async def fetch(session, url, i):
    print(i + 1)
    async with session.get(url) as response:
        html = await response.text()
        soup = BeautifulSoup(html, "html.parser")
        cont_thumb = soup.find_all("div", "cont_thumb")
        for cont in cont_thumb:
            title = cont.find("p", "txt_thumb")
            if title is not None:
                print(title.text)


async def main():
    BASE_URL = "https://bjpublic.tistory.com/category/%EC%A0%84%EC%B2%B4%20%EC%B6%9C%EA%B0%84%20%EB%8F%84%EC%84%9C"
    urls = [f"{BASE_URL}?page={i}" for i in range(1, 10)]
    async with aiohttp.ClientSession() as session:
        await asyncio.gather(*[fetch(session, url, i) for i, url in enumerate(urls)])


if __name__ == "__main__":
    asyncio.run(main())


In [ ]:
# magic variable = __name__
import aiohttp
import asyncio
import nest_asyncio
from config import get_secret
nest_asyncio.apply()

async def fetch(session, url, i):
    print(i + 1)
    headers = {
        'X-Naver-Client-ID':get_secret('X-Naver-Client-ID'),
        'X-Naver-Client-Secret':get_secret('X-Naver-Client-Secret')
    }
    async with session.get(url, headers=headers) as response:
        result = await response.json()
        items = result['items']
        images = [item['link'] for item in items]
        print(images)

async def main():
    BASE_URL = "https://openapi.naver.com/v1/search/image"
    keyword = 'cat'
    urls = [f'{BASE_URL}?query={keyword}&display=20&start={i}' for i in range(1,10)]
    async with aiohttp.ClientSession() as session:
        await asyncio.gather(*[fetch(session, url, i) for i, url in enumerate(urls)])



if __name__ == '__main__':
    asyncio.run(main())

In [ ]:
import os
import aiohttp
import asyncio
from config import get_secret
import aiofiles

# 함수 앞에 async를 붙인다.
async def img_downloader(session, img):
    img_name = img.split('/')[-1].split('?')[0]
    try :
        os.mkdir('./images')
    
    except FileExistsError as e:
        pass
    async with session.get(img) as response:
        if response.status == 200:
            async with aiofiles.open(f'./images/{img_name}', mode='wb') as file:
                img_data = await response.read()
                await file.write(img_data)
async def fetch(session, url, i):
    print(i+1)
    headers = {
        'X-Naver-Client-Id': get_secret('X-Naver-Client-ID'),
        'X-Naver-Client-Secret' : get_secret("X-Naver-Client-Secret"),
        
    }
    async with session.get(url, headers=headers) as response:
        result = await response.json()
        items = result['items']
        images = [item['link'] for item in items]
        
        # *을 붙이면 tuple 형태로 input이 된다. 
        await asyncio.gather(*[img_downloader(session, img) for img in images])
async def main():
    BASE_URL = "https://openapi.naver.com/v1/search/image"
    keyword = 'cat'
    urls = [f'{BASE_URL}?query={keyword}&display=20&start={1+i*20}' for i in range(10)]
    async with aiohttp.ClientSession() as session:
        await asyncio.gather(*[fetch(session, url, i) for i, url in enumerate(urls)])  
    
if __name__ == "__main__":
    asyncio.run(main())